<a href="https://colab.research.google.com/github/gabrielbribeiroo/OperationalResearch_UFPB/blob/main/MixProducao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula prática: Mix de Produção
<sup>Adaptado dos exercícios 2.3 e 2.5 do livro `Pesquisa Operacional`, de `Arenales, Armentano, Morabito e Yanasse`.</sup>

## Exercício 1

### Descrição do problema
Uma fundição tem de produzir 10 toneladas de um tipo de liga metálica e, para isso, tem disponível: lingotes de ferro, grafite e sucata. Dois componentes são relevantes para a liga: carbono e silício. As tabelas a seguir fornecem a fração, em termos percentuais, desses elementos nos ingredientes disponíveis, seus custos unitários, bem como a composição da liga (isto é, porcentagens mínima e máxima de cada componente da liga).

Frações dos elementos (%) nos ingredientes e custo dos ingredientes (R$/ton):

| | Lingotes | Grafite | Sucata |
|:---|:---:|:---:|:---:|
| Carbono | 0.5 | 90 | 9 |
| Silício | 14 | - | 27 |
| Custo | 90 | 180 | 25 |

Frações (%) mínima e máxima dos componentes na liga:

| | min | max |
|:---|:---:|:---:|
|Carbono | 0.0 | 9.5 |
|Silício | 19 | 20 |


Escreva um modelo de otimização linear para determinar as quantidades dos ingredientes para compor a liga metálica, de modo que as especificações técnicas sejam satisfeitas e o custo seja mínimo.

### Resolução

In [ ]:
# instalação e importação do pacote mip
!pip install mip

from mip import *

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.6/462.6 kB 26.6 MB/s eta 0:00:00
  Attempting uninstall: cffi
    Found existing installation: cffi 1.17.1
    Uninstalling cffi-1.17.1:
      Successfully uninstalled cffi-1.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pygit2 1.18.0 requires cffi>=1.17.0, but you have cffi 1.15.1 which is incompatible.


Carrega Dados

In [ ]:
# composição de cada ingrediente
a = {
    'l': {'c': 0.005, 's': 0.14}, # lingotes (carbono e silício)
    'g': {'c': 0.9,   's': 0.0}, # grafite (carbono e silício)
    's': {'c': 0.09,  's': 0.27}, # sucata (carbono e silício)
}

# custo
c = {'l': 90, 'g': 180, 's': 25} # lingotes, grafite, sucata (preço por tonelada)

# composições mínimas e máximas dos componentes
n = {'c': 0.0, 's': 0.19}  # min (carbono e silício)
m = {'c': 0.095, 's': 0.2} # max (carbono e silício)

# quantidade desejada da liga
Q = 10

Cria modelo

In [ ]:
model = Model(sense=MINIMIZE, solver_name=CBC) # implementa modelo

# criação/adição da variável no modelo
x = {i: model.add_var(var_type=CONTINUOUS, name=f'x_{i}', lb=0.0) for i in  ['l', 'g', 's']}

model.objective = c['l']*x['l'] + c['g']*x['g'] + c['s']*x['s'] # função objetivo (menor custo possível)

# soma dos ingredientes deve atingir a quantidade de liga desejada (10ton)
model += x['l'] + x['g'] + x['s'] == Q

# restrição de carbono na liga
carbono = x['l']*a['l']['c'] + x['g']*a['g']['c'] + x['s']*a['s']['c']
model += n['c']*Q <= carbono
model += carbono <= m['c']*Q

# restrição de silicio na liga
silicio = x['l']*a['l']['s'] + x['g']*a['g']['s'] + x['s']*a['s']['s']
model += n['s']*Q <= silicio
model += silicio <= m['s']*Q

model.write("model.lp") # salva modelo em arquivo
with open("model.lp") as f: # lê e exibe conteúdo do arquivo
  print(f.read())

\Problem name: 

Minimize
OBJROW: 90 x_l + 180 x_g + 25 x_s
Subject To
constr(0):  x_l + x_g + x_s = 10
constr(1):  0.00500 x_l + 0.90000 x_g + 0.09000 x_s >= -0
constr(2):  0.00500 x_l + 0.90000 x_g + 0.09000 x_s <= 0.95000
constr(3):  0.14000 x_l + 0.27000 x_s >= 1.90000
constr(4):  0.14000 x_l + 0.27000 x_s <= 2
Bounds
End



Executa

In [ ]:
def solve(model):
  status = model.optimize()

  print("Status = ", status)
  print(f"Solution value  = {model.objective_value:.2f}\n")

  print("Solution:")
  for v in model.vars:
      print(f"{v.name} = {v.x:.2f}")

solve(model)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 600.00

Solution:
x_l = 5.38
x_g = 0.00
x_s = 4.62


## Exercício 2

Agora considere que os ingredientes tem o estoque limitado, de acordo com a tabela abaixo.

| | Lingotes | Grafite | Sucata |
|:---|:---:|:---:|:---:|
| Estoque (ton) | 5 | 5 | 12 |

Como o modelo pode ser modificado para atender a esse requisito?

### Código

Carrega Dados

In [ ]:
# estoque
e = {'l': 5, 'g': 5, 's': 12}

Cria modelo

In [ ]:
model += x['l'] <= e['l']
model += x['g'] <= e['g']
model += x['s'] <= e['s']

model.write("modelo2.lp") # salva modelo em arquivo
with open("modelo2.lp") as f: # Lê e exibe conteúdo do arquivo
  print(f.read())

\Problem name: 

Minimize
OBJROW: 90 x_l + 180 x_g + 25 x_s
Subject To
constr(0):  x_l + x_g + x_s = 10
constr(1):  0.00500 x_l + 0.90000 x_g + 0.09000 x_s >= -0
constr(2):  0.00500 x_l + 0.90000 x_g + 0.09000 x_s <= 0.95000
constr(3):  0.14000 x_l + 0.27000 x_s >= 1.90000
constr(4):  0.14000 x_l + 0.27000 x_s <= 2
constr(5):  x_l <= 5
constr(6):  x_g <= 5
constr(7):  x_s <= 12
Bounds
End



Executa

In [ ]:
solve(model)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 603.70

Solution:
x_l = 5.00
x_g = 0.19
x_s = 4.81


## Exercício 3

Suponha agora que duas ligas metálicas devem ser preparadas e os mesmos ingredientes são utilizados em ambas. A liga especificada no Exercício 1 é referida como liga 1 e devem ser produzidas 10 toneladas desta liga. Da outra liga, referida como liga 2, devem ser produzidas 6 toneladas e suas composições mínima e máxima são dadas na tabela abaixo.

| | min | max |
|:---|:---:|:---:|
|Carbono | 0.00 | 40 |
|Silício | 12 | 19 |


### Código

Carrega dados

In [ ]:
# composições mínimas e máximas dos componentes
n = [{'c': 0.0, 's': 0.19}, {'c': 0.0, 's': 0.12}]
m = [{'c': 0.095, 's': 0.2}, {'c': 0.4, 's': 0.19}]

# quantidade desejada da liga
Q = [10, 6]

Cria modelo

In [ ]:
model = Model(sense=MINIMIZE, solver_name=CBC) # implementa modelo

# criação/adição da variável no modelo
x = [{i: model.add_var(name=f'x_{l}_{i}') for i in ['l', 'g', 's']} for l in range(2)]

# função objetivo
model.objective = c['l']*x[0]['l'] + c['g']*x[0]['g'] + c['s']*x[0]['s'] + c['l']*x[1]['l'] + c['g']*x[1]['g'] + c['s']*x[1]['s']

# a soma dos ingredientes usados na liga 0 deve ser igual a Q0
model += x[0]['l'] + x[0]['g'] + x[0]['s'] == Q[0]

# a soma dos ingredientes usados na liga 1 deve ser igual a Q1
model += x[1]['l'] + x[1]['g'] + x[1]['s'] == Q[1]

# restrição na quantidade de carbono na liga 0
carbono = x[0]['l']*a['l']['c'] + x[0]['g']*a['g']['c'] + x[0]['s']*a['s']['c']
model += n[0]['c']*Q[0] <= carbono
model += carbono <= m[0]['c']*Q[0]

# restrição na quantidade de carbono na liga 1
carbono = x[1]['l']*a['l']['c'] + x[1]['g']*a['g']['c'] + x[1]['s']*a['s']['c']
model += n[1]['c']*Q[1] <= carbono
model += carbono <= m[1]['c']*Q[1]

# restrição na quantidade de silicio na liga 0
silicio = x[0]['l']*a['l']['s'] + x[0]['g']*a['g']['s'] + x[0]['s']*a['s']['s']
model += n[0]['s']*Q[0] <= silicio
model += silicio <= m[0]['s']*Q[0]

# restrição na quantidade de silicio na liga 1
silicio = x[1]['l']*a['l']['s'] + x[1]['g']*a['g']['s'] + x[1]['s']*a['s']['s']
model += n[1]['s']*Q[1] <= silicio
model += silicio <= m[1]['s']*Q[1]

# restrições de estoque
model += x[0]['l'] + x[1]['l'] <= e['l']
model += x[0]['g'] + x[1]['g'] <= e['g']
model += x[0]['s'] + x[1]['s'] <= e['s']

model.write("modelo3.lp") # salva modelo em arquivo
with open("modelo3.lp") as f: # lê e exibe conteúdo do arquivo
  print(f.read())

\Problem name: 

Minimize
OBJROW: 90 x_0_l + 180 x_0_g + 25 x_0_s + 90 x_1_l + 180 x_1_g + 25 x_1_s
Subject To
constr(0):  x_0_l + x_0_g + x_0_s = 10
constr(1):  x_1_l + x_1_g + x_1_s = 6
constr(2):  0.00500 x_0_l + 0.90000 x_0_g + 0.09000 x_0_s >= -0
constr(3):  0.00500 x_0_l + 0.90000 x_0_g + 0.09000 x_0_s <= 0.95000
constr(4):  0.00500 x_1_l + 0.90000 x_1_g + 0.09000 x_1_s >= -0
constr(5):  0.00500 x_1_l + 0.90000 x_1_g + 0.09000 x_1_s <= 2.40000
constr(6):  0.14000 x_0_l + 0.27000 x_0_s >= 1.90000
constr(7):  0.14000 x_0_l + 0.27000 x_0_s <= 2
constr(8):  0.14000 x_1_l + 0.27000 x_1_s >= 0.72000
constr(9):  0.14000 x_1_l + 0.27000 x_1_s <= 1.14000
constr(10):  x_0_l + x_1_l <= 5
constr(11):  x_0_g + x_1_g <= 5
constr(12):  x_0_s + x_1_s <= 12
Bounds
End



Executa

In [ ]:
solve(model)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 1029.26

Solution:
x_0_l = 4.32
x_0_g = 0.51
x_0_s = 5.17
x_1_l = 0.68
x_1_g = 1.45
x_1_s = 3.87
